In [1]:
import gurobipy as gp
import math

In [2]:
m = gp.Model("Curtain")

Restricted license - for non-production use only - expires 2026-11-23


In [3]:
LENGTHS = [
    352,
    220,
    276,
    276,
    250,
    250,
]

In [4]:
TRACK_LENGTH = 140

In [5]:
def calculate_max_track_count(lengths, track_length):
    count = 0

    for l in lengths:
        count += math.ceil(l / track_length)

    return count

In [6]:
MAX_TRACKS = calculate_max_track_count(LENGTHS, TRACK_LENGTH)
MAX_TRACKS

13

In [7]:
print(
    f"We have {len(LENGTHS)} windows need to install curtains, the track's length is {TRACK_LENGTH}.\n" \
    f"And the windows' lengths are {','.join([f'{i}' for i in LENGTHS])}. So how can we minimize the tracks' usage.\n" \
    f"We also know that we can cutting and combine tracks, but we want to minimize the cutting numbers as well as the usage of tracks."
)


We have 6 windows need to install curtains, the track's length is 140.
And the windows' lengths are 352,220,276,276,250,250. So how can we minimize the tracks' usage.
We also know that we can cutting and combine tracks, but we want to minimize the cutting numbers as well as the usage of tracks.


In [8]:
x = m.addVars(MAX_TRACKS, len(LENGTHS), vtype=gp.GRB.INTEGER, name="x")
y = m.addVars(MAX_TRACKS, len(LENGTHS), vtype=gp.GRB.BINARY, name="y")
z = m.addVars(MAX_TRACKS, vtype=gp.GRB.BINARY, name="z")
w = m.addVars(MAX_TRACKS, vtype=gp.GRB.BINARY, name="w")

m.update()

In [9]:
# Constraint 0: All values greater than 0
for l in range(len(LENGTHS)):
    for t in range(MAX_TRACKS):
        m.addConstr(x[t, l] >= 0, f"non_negative_{t}_{l}")
        m.addConstr(x[t, l] <= TRACK_LENGTH * y[t, l], f"link_x_to_y_{t}_{l}")
        m.addConstr(x[t, l] >= y[t, l], f"link_y_to_x_{t}_{l}")

In [10]:
# Constraint 1: Curtain should have enough length
for i, l in enumerate(LENGTHS):
    m.addConstr(sum(x[t, i] for t in range(MAX_TRACKS)) == l, name=f"curtain_{i}_length")

In [11]:
# Constraint 2: Track should be full length

for t in range(MAX_TRACKS):
    m.addConstr(sum(x[t, l] for l in range(len(LENGTHS))) <= TRACK_LENGTH * z[t], name=f"track_{t}_capacity")




    m.addConstr(sum(x[t, l] for l in range(len(LENGTHS))) <= TRACK_LENGTH * w[t] + (TRACK_LENGTH - 1) * (1 - w[t]), f"track_{t}full_upper")
    m.addConstr(sum(x[t, l] for l in range(len(LENGTHS))) >= TRACK_LENGTH * w[t], name=f"track_{t}_full_lower")

    m.addConstr(w[t] <= z[t], f"track_{t}_usage_link")







In [12]:
# Objective 1: Minimize tracks
m.setObjectiveN(sum(z[t] for t in range(MAX_TRACKS)), index=0, priority=20)


In [13]:
# Objective 2: Minimize comibination

m.setObjectiveN(sum(y[t, l] for t in range(MAX_TRACKS) for l in range(len(LENGTHS))), index=1, priority=10)

In [14]:
# Objective 3: Minimize cuts
cuts = m.addVars(MAX_TRACKS, vtype=gp.GRB.INTEGER, name="custs")

for t in range(MAX_TRACKS):
    m.addConstr(cuts[t] == sum(y[t, l] for l in range(len(LENGTHS))) - z[t] + z[t] - w[t], f"cut_count_{t}")
    m.addConstr(cuts[t] >= 0, f"cut_count_non_negative_{t}")

m.setObjectiveN(sum(cuts[t] for t in range(MAX_TRACKS)), index=2, priority=5)

In [15]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (armlinux64 - "Debian GNU/Linux 11 (bullseye)")

CPU model: ARM64
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 318 rows, 195 columns and 884 nonzeros
Model fingerprint: 0xd247e475
Variable types: 0 continuous, 195 integer (104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 4e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve...
---------------------------------------------------------------------------

Presolve removed 91 rows and 0 columns
Presolve time: 0.00s
Presolved: 227 rows, 195 columns, 793 nonzeros

Multi-objectives: optimize objective 1 () ...
-----

In [16]:
for t in range(MAX_TRACKS):
    for l in range(len(LENGTHS)):
        if x[t, l].X > 0:
            print(f"Track {t} for Curtain {l}, with length: {x[t, l].X}")

Track 0 for Curtain 4, with length: 140.0
Track 1 for Curtain 2, with length: 140.0
Track 2 for Curtain 0, with length: 140.0
Track 3 for Curtain 3, with length: 140.0
Track 4 for Curtain 5, with length: 140.0
Track 5 for Curtain 3, with length: 136.0
Track 6 for Curtain 2, with length: 136.0
Track 8 for Curtain 0, with length: 72.0
Track 8 for Curtain 5, with length: 68.0
Track 9 for Curtain 4, with length: 110.0
Track 10 for Curtain 0, with length: 140.0
Track 11 for Curtain 1, with length: 80.0
Track 11 for Curtain 5, with length: 42.0
Track 12 for Curtain 1, with length: 140.0


In [17]:
for l in range(len(LENGTHS)):
    length = LENGTHS[l]
    print(f"Curtain {l} should have length: {length}")

    sum_length = 0
    for t in range(MAX_TRACKS):
        if x[t, l].X > 0:
            sum_length += x[t, l].X
            print(f"track {t}: {x[t, l].X}", end=" ")
    print(f" => {sum_length}", end="")

    print("")
    print("-" * 32)

Curtain 0 should have length: 352
track 2: 140.0 track 8: 72.0 track 10: 140.0  => 352.0
--------------------------------
Curtain 1 should have length: 220
track 11: 80.0 track 12: 140.0  => 220.0
--------------------------------
Curtain 2 should have length: 276
track 1: 140.0 track 6: 136.0  => 276.0
--------------------------------
Curtain 3 should have length: 276
track 3: 140.0 track 5: 136.0  => 276.0
--------------------------------
Curtain 4 should have length: 250
track 0: 140.0 track 9: 110.0  => 250.0
--------------------------------
Curtain 5 should have length: 250
track 4: 140.0 track 8: 68.0 track 11: 42.0  => 250.0
--------------------------------


In [18]:

# 打印结果：轨道使用情况
print("\n轨道使用情况:")
for t in range(MAX_TRACKS):
    if z[t].X > 0.5:  # 如果轨道t被使用
        print(f"轨道 {t}:")
        segments = []
        for l in range(len(LENGTHS)):
            if x[t, l].X > 0:
                segments.append(f"{x[t, l].X} 厘米给窗帘 {l}")
        
        total_length = sum(x[t, l].X for l in range(len(LENGTHS)))
        usage_status = "完整使用" if w[t].X > 0.5 else f"部分使用({total_length}/{TRACK_LENGTH})"
        
        print(f"  - {', '.join(segments)} [{usage_status}]")

# 打印每个窗帘的轨道分配明细
print("\n窗帘分配详情:")
for l in range(len(LENGTHS)):
    print(f"窗帘 {l} (长度: {LENGTHS[l]} 厘米):")
    total = 0
    for t in range(MAX_TRACKS):
        if x[t, l].X > 0:
            print(f"  - 从轨道 {t} 取 {x[t, l].X} 厘米")
            total += x[t, l].X
    print(f"  总计: {total} 厘米")

# 打印总结
used_tracks = sum(1 for t in range(MAX_TRACKS) if z[t].X > 0.5)
cross_curtain_cuts = sum(max(0, sum(1 for l in range(len(LENGTHS)) if y[t, l].X > 0.5) - 1) for t in range(MAX_TRACKS))

print(f"\n总结：使用了 {used_tracks} 个轨道") #，需要总共 {total_cuts} 次切割")
print(f"其中：{int(cross_curtain_cuts)} 次跨窗帘切割") #，{int(length_cuts)} 次因轨道未用完整长度的切割")

length_cuts = sum(1 for t in range(MAX_TRACKS) if z[t].X > 0.5 and w[t].X < 0.5)
total_cuts = int(sum(cuts[t].X for t in range(MAX_TRACKS)))


print(f"\n总结：使用了 {used_tracks} 个轨道，需要总共 {total_cuts} 次切割")
print(f"其中：{int(cross_curtain_cuts)} 次跨窗帘切割，{int(length_cuts)} 次因轨道未用完整长度的切割")


轨道使用情况:
轨道 0:
  - 140.0 厘米给窗帘 4 [完整使用]
轨道 1:
  - 140.0 厘米给窗帘 2 [完整使用]
轨道 2:
  - 140.0 厘米给窗帘 0 [完整使用]
轨道 3:
  - 140.0 厘米给窗帘 3 [完整使用]
轨道 4:
  - 140.0 厘米给窗帘 5 [完整使用]
轨道 5:
  - 136.0 厘米给窗帘 3 [部分使用(136.0/140)]
轨道 6:
  - 136.0 厘米给窗帘 2 [部分使用(136.0/140)]
轨道 8:
  - 72.0 厘米给窗帘 0, 68.0 厘米给窗帘 5 [完整使用]
轨道 9:
  - 110.0 厘米给窗帘 4 [部分使用(110.0/140)]
轨道 10:
  - 140.0 厘米给窗帘 0 [完整使用]
轨道 11:
  - 80.0 厘米给窗帘 1, 42.0 厘米给窗帘 5 [部分使用(122.0/140)]
轨道 12:
  - 140.0 厘米给窗帘 1 [完整使用]

窗帘分配详情:
窗帘 0 (长度: 352 厘米):
  - 从轨道 2 取 140.0 厘米
  - 从轨道 8 取 72.0 厘米
  - 从轨道 10 取 140.0 厘米
  总计: 352.0 厘米
窗帘 1 (长度: 220 厘米):
  - 从轨道 11 取 80.0 厘米
  - 从轨道 12 取 140.0 厘米
  总计: 220.0 厘米
窗帘 2 (长度: 276 厘米):
  - 从轨道 1 取 140.0 厘米
  - 从轨道 6 取 136.0 厘米
  总计: 276.0 厘米
窗帘 3 (长度: 276 厘米):
  - 从轨道 3 取 140.0 厘米
  - 从轨道 5 取 136.0 厘米
  总计: 276.0 厘米
窗帘 4 (长度: 250 厘米):
  - 从轨道 0 取 140.0 厘米
  - 从轨道 9 取 110.0 厘米
  总计: 250.0 厘米
窗帘 5 (长度: 250 厘米):
  - 从轨道 4 取 140.0 厘米
  - 从轨道 8 取 68.0 厘米
  - 从轨道 11 取 42.0 厘米
  总计: 250.0 厘米

总结：使用了 12 个轨道
其中：2 次跨窗帘切割

总结：使用了 12 个

In [19]:
for t in range(MAX_TRACKS):
    used_length = sum(x[t, l].X for l in range(len(LENGTHS)))
    is_used = z[t].X > 0.5
    is_full_used = z[t].X > 0.5 and w[t].X > 0.5

    print_out = f"track {t} is used {used_length}"
    if is_used:
        if is_full_used:
            print_out += " (full)"
        else:
            print_out += " (partily)"

        for l in range(len(LENGTHS)):
            if y[t, l].X > 0.5:
                print_out += f"\n - curtain {l}: {x[t, l].X}"
    else:
        print_out += " (unused)"
    print(print_out)


track 0 is used 140.0 (full)
 - curtain 4: 140.0
track 1 is used 140.0 (full)
 - curtain 2: 140.0
track 2 is used 140.0 (full)
 - curtain 0: 140.0
track 3 is used 140.0 (full)
 - curtain 3: 140.0
track 4 is used 140.0 (full)
 - curtain 5: 140.0
track 5 is used 136.0 (partily)
 - curtain 3: 136.0
track 6 is used 136.0 (partily)
 - curtain 2: 136.0
track 7 is used 0.0 (unused)
track 8 is used 140.0 (full)
 - curtain 0: 72.0
 - curtain 5: 68.0
track 9 is used 110.0 (partily)
 - curtain 4: 110.0
track 10 is used 140.0 (full)
 - curtain 0: 140.0
track 11 is used 122.0 (partily)
 - curtain 1: 80.0
 - curtain 5: 42.0
track 12 is used 140.0 (full)
 - curtain 1: 140.0


In [20]:
for t in range(MAX_TRACKS):
    print(f"track {t} is used {z[t].X} and {w[t].X}")

track 0 is used 1.0 and 1.0
track 1 is used 1.0 and 1.0
track 2 is used 1.0 and 1.0
track 3 is used 1.0 and 1.0
track 4 is used 1.0 and 1.0
track 5 is used 1.0 and -0.0
track 6 is used 1.0 and -0.0
track 7 is used -0.0 and -0.0
track 8 is used 1.0 and 1.0
track 9 is used 1.0 and -0.0
track 10 is used 1.0 and 1.0
track 11 is used 1.0 and 0.0
track 12 is used 1.0 and 1.0


In [21]:
# Save optimization results to JSON
import json

result = {
    "track_length": TRACK_LENGTH,
    "max_tracks": MAX_TRACKS,
    "curtain_lengths": LENGTHS,
    "track_allocations": [],
    "stats": {
        "used_tracks": 0,
        "total_cuts": 0
    }
}

# Extract track allocations
for t in range(MAX_TRACKS):
    for l in range(len(LENGTHS)):
        if x[t, l].X > 0:
            result["track_allocations"].append({
                "track_idx": t,
                "curtain_idx": l,
                "length": int(x[t, l].X)
            })

# Calculate statistics
used_tracks = sum(1 for t in range(MAX_TRACKS) if z[t].X > 0.5)
total_cuts = sum(cuts[t].X for t in range(MAX_TRACKS))

result["stats"]["used_tracks"] = used_tracks
result["stats"]["total_cuts"] = total_cuts

# Save to JSON file
with open('curtain_results.json', 'w') as f:
    json.dump(result, f, indent=2)

print(f"Results saved to curtain_results.json")

Results saved to curtain_results.json
